In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import datetime
from dateutil.relativedelta import relativedelta

from cartopy import crs as ccrs, feature as cfeature
import warnings
warnings.filterwarnings('ignore')

import netCDF4
from netCDF4 import Dataset

import xarray as xr
import glob, os
#print(xr.__version__)

## Just the exact same as the OHC files. B/c it's also ORAS5 data.

In [2]:
os.chdir("/d6/bxw2101/oras5_sit")
sit_1 = glob.glob("sit_firstgroup/iicethic_control*.nc")
sit_2 = glob.glob("sit_secondgroup/iicethic_control*.nc")
all_sit_files = sit_1 + sit_2
all_sit_files.sort()
print("# of monthly files: " + str(len(all_sit_files)))

# of monthly files: 521


In [ ]:
# ds_all = xr.open_mfdataset(all_sit_files, concat_dim='time_counter', combine='nested')
# #this takes maybe 20-30s

In [ ]:
# ds_crop = ds_all.where(ds_all.nav_lat < -50)
# ds_crop.to_netcdf('all_sit_monthly.nc', mode='w',format='NETCDF4')
# #this also took maybe 20-30s

In [3]:
filename = '/d6/bxw2101/oras5_sit/all_sit_monthly.nc'
ds = xr.open_dataset(filename, decode_times=False)

date_1 = datetime.datetime.strptime("01/16/1979", "%m/%d/%Y")
def convert_times(x):
  return (date_1 + relativedelta(hours=int(x))).replace(day=1, hour=0)
convert_times_v = np.vectorize(convert_times)
ds = ds.assign_coords(time_counter=convert_times_v(ds.time_counter))
ds = ds.rename({'time_counter': 'tdim'})

# Dataset notes:
# y: indices 0 to 1021. x: indices 0 to 1441.
# for nav_lat and nav_lon: it's in the format of (y, x). origin is bottom left.
# longitude, AFAIK, roughly increases by 0.25 degrees each increment. roughly.
# I have already cropped using .where() to just include latitudes < -50 degrees for the southern ocean.

In [ ]:
# Just plotting nav_lon and nav_lat. they are two-dimensional scalar fields. (obv)
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(14, 4))
ds.nav_lon.plot(ax=ax1)
ds.nav_lat.plot(ax=ax2)

## Quick n dirty SIT and OHC comparison w the WHOLE coordinate system

In [5]:
filename = '/d6/bxw2101/oras5_ohc/all_ohc_monthly.nc'
ds_ohc = xr.open_dataset(filename, decode_times=False)

date_1 = datetime.datetime.strptime("01/16/1978", "%m/%d/%Y")
def convert_times(x):
  return (date_1 + relativedelta(hours=int(x))).replace(day=1)
convert_times_v = np.vectorize(convert_times)
ds_ohc = ds_ohc.assign_coords(time_counter=convert_times_v(ds_ohc.time_counter))
ds_ohc = ds_ohc.rename({'time_counter': 'tdim'})
ds_ohc = ds_ohc.drop_isel(tdim=list(range(12)))
# Dataset notes:
# y: indices 0 to 1021. x: indices 0 to 1441.
# for nav_lat and nav_lon: it's in the format of (y, x). origin is bottom left.
# longitude, AFAIK, roughly increases by 0.25 degrees each increment. roughly.
# I have already cropped using .where() to just include latitudes < -50 degrees for the southern ocean.

In [ ]:
# ds_ohc.sohtc300
# ds.iicethic
og_grid_corr = xr.corr(ds.iicethic, ds_ohc.sohtc300, dim='tdim') # takes some time
#og_grid_corr.min()
#og_grid_corr.max()

### WOAH. here I am doing the correlation between SIT and OHC absolute values, not the anomalies. 
Should I be doing correlation btwn absolute values or correlations between ANOMALIES???

In [ ]:
sp = ccrs.SouthPolarStereo()
pc = ccrs.PlateCarree() # The OG Data is in pc projection.

fig = plt.figure(figsize=(10,7))
ax = plt.axes(projection=sp)
og_grid_corr.plot.pcolormesh(
    ax=ax, transform=pc, x="nav_lon", y="nav_lat", vmin = -1, vmax = .3, cmap='plasma' # This gon be the way we plot this stuff.
)
ax.coastlines()
ax.set_extent([-3950000., 3950000., 4350000., -3950000.], crs=sp)
plt.title("Correlation btwn SIT and OHC, using the original fine coordinates")

In [ ]:
# just tryna see one single frame:

arb_time = ds.iicethic.sel(tdim='2017-11-01')
fig = plt.figure(figsize=(10,7))
ax = plt.axes(projection=sp)
arb_time[0].plot.pcolormesh(
    ax=ax, transform=pc, x="nav_lon", y="nav_lat", # This gon be the way we plot this stuff.
)
ax.coastlines()
ax.set_extent([-3950000., 3950000., 4350000., -3950000.], crs=sp)
plt.title("Correlation btwn SIT and OHC, using the original fine coordinates")

## Tryna build a new SIT dataset that is 2x2 lon/lat.

In [6]:
# Building the new dataset takes a while. There are a bunch of lat/lon combinations to go through.
# This code takes the NEAREST

# tdim: 516, Y: 20, X: 180 IS OUR DIMENSIONS. (for 1979-01-01 to 2021-12-01)
lat_bins = np.arange(-88., -49, 2)
lon_bins = np.arange(0., 359, 2)
building_data = np.empty([20, 180, 521])
#y, x, tdim is our order of dimensions.

for lat in lat_bins:
    for lon in lon_bins:
        # First, find the index of the grid point nearest a specific lat/lon.
        
        # convert 180 to 358 to -180 to -2.
        if lon > 179:
            lon = lon - 360
        
        abslat = np.abs(ds.nav_lat-lat)
        abslon = np.abs(ds.nav_lon-lon)

        c = abslon**2 +  abslat**2

        (ypts, xpts) = np.where(c == np.min(c))
        yloc = ypts[0]
        xloc = xpts[0]

        # Now I can use that index location to get the values at the x/y diminsion
        point_ds = ds.isel(x=xloc, y=yloc)
        point_ds = point_ds.assign_coords({"y": lat, "x": lon})
        #print(point_ds)

        yi = int((lat + 88) / 2)
        xi = int(lon / 2)

        building_data[yi][xi] = point_ds.iicethic.values

In [7]:
times = ds.tdim.values
regridded_sit = xr.DataArray(building_data, coords=[lat_bins, lon_bins, times], dims=['y', 'x', 'tdim'])
new_ds = xr.Dataset(data_vars = {"sit": regridded_sit})
new_ds = new_ds.transpose("tdim", "y", "x")
new_ds.to_netcdf('/d6/bxw2101/combined_netcdf_files/new_sit_monthly_2x2.nc', mode='w',format='NETCDF4')
# regridded_ohc here is the new!

## doing stuff w the 2x2 latitude longitude grid. PLOT LIKE THE SST FILES. Calculating the anomaly here.

In [9]:
filename = '/d6/bxw2101/combined_netcdf_files/new_sit_monthly_2x2.nc'
r_ds = xr.open_dataset(filename)
r_sit = r_ds.sit

In [ ]:
# NEW REGRIDDED SIT plot
arb_time_regrid = r_sit.sel(tdim='2017-11-01')
fig = plt.figure(figsize=(10,7))
ax = plt.axes(projection=sp)
arb_time_regrid.plot(transform=pc) #arb_time.plot(ax=ax,  vmin=0, vmax=1, cmap='coolwarm')
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.set_extent([-3950000., 3950000., 4350000., -3950000.], crs=sp)

In [11]:
r_clim_period = r_ds.sel(tdim=slice('1979-01-01', '2021-12-01'))
r_clim_sit = r_clim_period.sit

r_sit_mon = r_sit.groupby('tdim.month')
r_sit_clim = r_clim_sit.groupby('tdim.month').mean(dim='tdim')
r_sit_anom = r_sit_mon - r_sit_clim
r_sit_anom = r_sit_anom.drop_vars('month')

In [12]:
sit_anom_ds = xr.Dataset(data_vars = {"sit_anom": r_sit_anom})
sit_anom_ds = sit_anom_ds.sel(tdim=slice('1979-01-01', '2021-12-01'))
sit_anom_ds.to_netcdf('/d6/bxw2101/combined_netcdf_files/sit_anom_monthly_2x2.nc', mode='w',format='NETCDF4')
#YAY WE HAVE THE OHC ANOMALY.